# import libraries

In [ ]:
import numpy as np
import pandas as pd

import gc

from IPython.display import clear_output
!pip install autokeras
clear_output()

import tensorflow as tf
from sklearn.model_selection import train_test_split
import autokeras as ak

# global variables

In [ ]:
ID = "Id"
TARGET = "Cover_Type"
TEST_SIZE = 0.2
RANDOM_SEED = 42
MAX_TRIAL = 3 # for simple test 
EPOCHS = 5  # for simple test 
VALIDATION_SPLIT = 0.15

# load data 

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-dec-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv")

# delete unnecessary columns

In [ ]:
train = train.drop([ID],axis=1)
test = test.drop([ID],axis=1)

# split data (input data and target data)

In [ ]:
y = train[TARGET]
X = train.drop([TARGET],axis=1)
X_test = test

gc.collect()

# scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
total = pd.concat([X,X_test],axis=0)
trans = StandardScaler()
trans.fit(total)

X= trans.transform(X)
X_test = trans.transform(X_test)

gc.collect()

# split data (train set and validation set)

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=TEST_SIZE,random_state=RANDOM_SEED)

# search best model and build best model 

In [ ]:
clf = ak.StructuredDataClassifier(max_trials=MAX_TRIAL,seed=RANDOM_SEED)
clf.fit(X_train, y_train, epochs=EPOCHS, validation_split=VALIDATION_SPLIT)

# evaluate model

In [ ]:
clf.evaluate(X_val, y_val)

# get best model

In [ ]:
model = clf.export_model()
model.summary()

# predict test data using best model

In [ ]:
pred_test = model.predict(X_test)
pred_test[:5]

# submission

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')
sub[TARGET]=(pred_test - 0.5).astype(int) + 1
sub.to_csv('sub.csv',index=False)
sub.head()